In [2]:
'''第二、三次檔案處理合併'''
'''目標：修改col名稱、補(surise、sunset)col空值'''
'''目標：挑選出 airport_info 中機場所對應的氣象資料'''
import os
import pandas as pd

# 建立挑選的機場 WBAN 清單
airport = pd.read_csv("C:/airport_info_0325.csv",dtype=object)
WBAN = list(airport["WBAN"].dropna(axis=0,how='any'))
WBAN_list = []
for i in WBAN:
    WBAN_list.append(i.zfill(5))

# 原資料夾路徑
folder_path ='C:/02_fp_data_weather1/'
# 新資料夾路徑
new_folder_path ='C:/02_fp_data_weather2/'
# 如果新資料夾不存在，則建立它
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

for file in WBAN_list:
    if os.path.exists(f"{folder_path}weather_info_{file}.csv"):
        # 讀取 CSV 檔案
        df = pd.read_csv(os.path.join(folder_path, f"weather_info_{file}.csv"), index_col=False)
    
        # 欄位切割、重新更名
        df['WBAN'] = df['STATION'].astype("string").str[-5:]
        df[['yyyy-mm-dd','hh:mm:ss']] = df['DATE'].str.split(pat='T',expand = True)
        df[['date_year','date_month','date_date']] = df['yyyy-mm-dd'].str.split(pat='-',expand = True)
        df[['time_hr','time_min','time_sec']] = df['hh:mm:ss'].str.split(pat=':',expand = True)
        df['dry_bulb_temp'] = df['HourlyDryBulbTemperature']
        df['wet_bulb_temp'] = df['HourlyWetBulbTemperature']
        df['dew_point_temp'] = df['HourlyDewPointTemperature']
        df['precipitation'] = df['HourlyPrecipitation']
        df['snow_fall'] = df['DailySnowfall']
        df['snow_depth'] = df['DailySnowDepth']
        df['relative_humidity'] = df['HourlyRelativeHumidity']
        df['wind_dict'] = df['HourlyWindDirection']
        df['wind_speed'] = df['HourlyWindGustSpeed']
        df['station_pressure'] = df['HourlyStationPressure']
        df['visibility'] = df['HourlyVisibility']

        # 針對 Sunrise、Sunset 進行處裡
        # 依照時間序列法(Time-Series Specific Methods)中【下個觀察值推估法(Next Observation Carried Backward(NOCB))】，補 surise、sunset 缺失值
        df['Sunrise'] = df['Sunrise'].fillna(method='bfill')
        df['Sunset'] = df['Sunset'].fillna(method='bfill')
        df['Sunrise'] = df['Sunrise'].fillna(method='ffill')
        df['Sunset'] = df['Sunset'].fillna(method='ffill')
        # 轉換型態
        df['Sunrise'] = df['Sunrise'].astype(int)
        df['Sunset'] = df['Sunset'].astype(int)
        # 切割欄位
        df['sunrise_hr'] = df['Sunrise'].astype("string").str[-4:-2]
        df['sunrise_min'] = df['Sunrise'].astype("string").str[-2:]
        df['sunset_hr'] = df['Sunset'].astype("string").str[-4:-2]
        df['sunset_min'] = df['Sunset'].astype("string").str[-2:]

        # 新資料的保留欄位
        cols_name = ['WBAN',
                    'date_year',
                    'date_month',
                    'date_date',
                    'time_hr',
                    'time_min',
                    'time_sec',
                    'sunrise_hr',
                    'sunrise_min',
                    'sunset_hr',
                    'sunset_min',
                    'dry_bulb_temp',
                    'wet_bulb_temp',
                    'dew_point_temp',
                    'precipitation',
                    'snow_fall',
                    'snow_depth',
                    'relative_humidity',
                    'wind_dict',
                    'wind_speed',
                    'station_pressure',
                    'visibility']

        # 複寫 df
        df = df[cols_name]

        # 另存新檔
        df.to_csv(os.path.join(new_folder_path, f"weather_info_{file}.csv"), index=False)